Data Wrangling Notebook for Famoso 2017 Supplemental Data 1
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [2]:
import pandas as pd
import numpy as np
import uuid

Silence unnecessary errors

In [3]:
try:
    import warnings
    warnings.filterwarnings('ignore')
except:
    pass

Import Famoso 2017 data

In [4]:
data = pd.read_csv("../Original_Data/Famoso_2017_Supplemental_Data_1.csv")

A lot of information is missing from the dataset and probably needs to be mined in the paper. Looking for things like event date, year collected, institution(?). 

In [5]:
# Questions

# Meaning of Formation
# Meaning of upper or lower turtle cover member 
# Meaning of geological submit
# Meaning of upper or lower
# Would notes just fit in with dynamicProperties?
# Method of searching for additional information 
# Is it correct to assume that basisOfRecord is Fossil Specimen?
# How to handle the trait terms
# Is specimen # the same as individualID?
# Current state of geome 

#eventID 
#country 
#locality 
#yearCollected 
#samplingProtocol 
#materialSampleID 
#basisOfRecord 
#scientificName 
#diagnosticID 
#measurementMethod 
#measurementUnit 
#measurementType
#measurementValue 

Process scientificName and verbatimScientificName 

In [6]:
# Create verbatimScientificName (Leave out of final dataset for now)
data = data.assign(verbatimScientificName = data['Genus'] + " " + data['species'])
 
# Create scientificName from verbatimScientificName data and fill nan with "Unknown"
data = data.assign(scientificName = data["verbatimScientificName"]).fillna("Unknown")

Adding additional required GEOME columns

In [7]:
data=data.assign(samplingProtocol="Unknown", measurementMethod="Unknown", basisOfRecord="FossilSpecimen")

Rearrange columns so that template columns are first, followed by measurement values

In [8]:
# Create column list
cols = data.columns.tolist()

# Specify desired columns
cols = ['Specimen #',
        'Locality',
        'notes',
        'scientificName',
        'samplingProtocol',
        'measurementMethod',
        'basisOfRecord']

# Subset dataframe
data = data[cols]

Matching template and column terms

In [9]:
# Renaming columns 
data = data.rename(columns = {'Specimen #': 'individualID',
                              'Locality':'verbatimLocality',
                              'notes':'dynamicProperties'})

Create necessary materialSampleID column and populate with UUID (use hex to remove dashes). Create necessary eventID column and populate with materialSampleID column

In [10]:
data = data.assign(materialSampleID = '')
data['materialSampleID'] = [uuid.uuid4().hex for _ in range(len(data.index))]

data = data.assign(eventID = data["materialSampleID"])

Create a long version of the data frame

In [11]:
# Creating long version, first specifiying keep variables, then naming variable and value
longVers = pd.melt(data, 
                id_vars = ['individualID',
                           'verbatimLocality',
                           'dynamicProperties',
                           'scientificName',
                           'samplingProtocol',
                           'measurementMethod',
                           'basisOfRecord'], 
                            var_name = 'measurementType', 
                            value_name = 'measurementValue')

Create diagnosticID which is a unique number for each measurement

In [12]:
longVers = longVers.assign(diagnosticID = '')
longVers['diagnosticID'] = np.arange(len(longVers))

If measurement value equals N/a, delete entire row

In [ ]:
longVers = longVers.dropna(subset = ['measurementValue'])

Writing long data csv file

In [ ]:
longVers.to_csv('../Mapped_Data/[].csv', index = False)